## 1.构建网络 初探

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

### 1.1构建模型

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)#单通道、输出通道（叠几层）、卷积核大小
        self.fc1 = nn.Linear(1350, 10)#in, out
    
    def forward(self, x):
        #x: [1,1,32,32]
        x = self.conv1(x) #[1,6,30,30]
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2)) #池化过程不用重叠[1,6,15,15]
        x = F.relu(x)
        x = x.view(x.size()[0], -1) #压到[1,1350]
        x = self.fc1(x)
        return x
    
model = CNN()
model

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)

### 1.2查看模型

模型的参数值，shape，grad

In [21]:
print(model.conv1, '\n')
print(model.conv1.weight.shape) 
print(model.conv1.bias.shape)
model.conv1.bias.grad

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1)) 

torch.Size([6, 1, 3, 3])
torch.Size([6])


tensor([ 1.2537,  0.0000,  0.1860,  0.0000, -0.6709,  0.8827])

In [23]:
print(model.fc1, '\n')
print(model.fc1.weight.shape) 
print(model.fc1.bias.shape)

Linear(in_features=1350, out_features=10, bias=True) 

torch.Size([10, 1350])
torch.Size([10])


查看所有parameters

In [3]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[[[ 0.2295, -0.2769,  0.1879],
          [-0.2160,  0.1160, -0.2031],
          [ 0.1784,  0.1745, -0.2437]]],


        [[[ 0.0662, -0.3169, -0.2802],
          [ 0.1660, -0.0944, -0.1504],
          [-0.1538, -0.3028,  0.0973]]],


        [[[ 0.0037,  0.2246,  0.1134],
          [-0.3086, -0.3174,  0.2631],
          [ 0.0581,  0.1560,  0.0604]]],


        [[[ 0.2531, -0.0741, -0.2685],
          [-0.1918, -0.2093,  0.0591],
          [-0.0220,  0.1060,  0.0461]]],


        [[[ 0.2819,  0.2070, -0.3130],
          [ 0.2241,  0.1345, -0.2879],
          [ 0.1541,  0.0086,  0.3181]]],


        [[[-0.1663, -0.0152,  0.2770],
          [-0.2049,  0.2131, -0.2883],
          [ 0.0913, -0.0793, -0.0869]]]], requires_grad=True)
Parameter containing:
tensor([ 0.2182, -0.0834,  0.0735, -0.2827,  0.1928,  0.0721],
       requires_grad=True)
Parameter containing:
tensor([[-0.0009,  0.0145, -0.0236,  ..., -0.0229,  0.0258, -0.0006],
        [ 0.0001,  0.0154, -0

In [4]:
for name,parameters in model.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


### 1.3BP

In [5]:
x = torch.rand(1,1,32,32)
y_pred = model(x)
y_pred.size()

torch.Size([1, 10])

In [6]:
model.zero_grad()

# y_pred.backward(torch.ones(y_pred.shape)) #传1进去，才能对向量backward

# 但是其实y_pred本身进行BP没意义，我们要的是loss来BP
# 或者 y_pred.mean().backward()

In [7]:
y_label = torch.arange(0,10).view(1,10).float()
loss_fn = nn.MSELoss()
loss = loss_fn(y_label, y_pred)
loss

tensor(28.7616, grad_fn=<MeanBackward0>)

In [8]:
optim = torch.optim.SGD(model.parameters(), lr=0.001)
optim.zero_grad()
loss.backward()
optim.step()

## 2.实战：图像分类器

In [114]:
a=torch.FloatTensor([[1],[0],[3]])
b=torch.FloatTensor([[0],[2],[3]])
print(a.eq(0))#找到等于0的 
print(a.shape) 

print(a.unsqueeze(0))
print(a.unsqueeze(0).shape)
# print(a.unsqueeze(0).expand(2,3,2,1))
a.unsqueeze(0).expand(2,3,5)#.shape

tensor([[0],
        [1],
        [0]], dtype=torch.uint8)
torch.Size([3, 1])
tensor([[[1.],
         [0.],
         [3.]]])
torch.Size([1, 3, 1])


tensor([[[1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.],
         [3., 3., 3., 3., 3.]],

        [[1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.],
         [3., 3., 3., 3., 3.]]])

In [111]:
# [1 for _ in range(5)]
[1]*5

[1, 1, 1, 1, 1]

In [112]:
x = torch.Tensor([[1], [2], [2]])
print(x,'\n',x.shape)
x.expand(3, 3)#.shape

tensor([[1.],
        [2.],
        [2.]]) 
 torch.Size([3, 1])


tensor([[1., 1., 1.],
        [2., 2., 2.],
        [2., 2., 2.]])